In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
import spacy
from spacy import displacy
from termcolor import colored
from extraction import findSVOs
import networkx as nx
import matplotlib.pyplot as plt
from topic_modelling import get_topics
import time
from oops import actionTaken, getViolations, getViolators

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [10]:
def identify_crime(text, types, folders):
    text = text.lower()
    data = []
    for i in range(len(types)):
        if(text.find(types[i]) != -1):
            data.append(folders[i])
    return data

In [17]:
df = pd.read_csv('data.csv')

df.Text = df.Text.apply(lambda x : x.replace('\\n', '\n'))
df.Text = df.Text.apply(lambda x : x.replace('\\t', '\t'))

types = ['insider', 'misappropriate', 'unregister', 'fraud']
folders = ['Insider Trading', 'Misappropriated Funds', 'Unregistered', 'Fraud']
crimes = []
actions = []
violations = []
violators = []

for i in range(len(df['Text'])):
    text = df['Text'][i]
    crimes.append(identify_crime(text, types, folders))
    action = actionTaken(text)
    violation = getViolations(text)
    violator = getViolators(text)
    violations.append(violation)
    violators.append(violator)
    actions.append(action)

df['Crimes'] = crimes
df['Violations'] = violations
df['Violators'] = violators
df['Action Taken'] = actions

In [18]:
df.head()

,Text,Crimes,Violations,Violators,Action Taken
0,sec charges at&t and three executives with sel...,[],[violations of the disclosure provisions of Se...,"[at&t, inc., et al.]",[the complaint seeks permanent injunctive reli...
1,u.s. securities and exchange commission\n\nlit...,"[Insider Trading, Fraud]",[violating Section 10(b) of the securities exc...,"[mitchel s. guttenberg, erik r. franklin, davi...",[]
2,u.s. securities and exchange commission\n\nlit...,[Insider Trading],[violating Sections 10(b) and 14(e) of the sec...,"[dr. bobby v. khan, civil cas]",[the commission seeks an order permanently enj...
3,\t\nu.s. securities and exchange commission\n\...,[Insider Trading],[violated Sections 10(b) and 14(e) of the secu...,"[matthew h. kluger and garrett d. bauer, cas]",[]
4,\t\nu.s. securities and exchange commission\n\...,[Insider Trading],[violations of Section 10(b) of the securities...,[],[without admitting or denying the allegations ...


In [19]:
df.Text = df.Text.apply(lambda x : x.replace('\n', '\\n'))
df.Text = df.Text.apply(lambda x : x.replace('\t', '\\t'))

df.to_csv('data.csv', index=False)